Notebook to perform sensitivity analysis on either direct simulations or emulations

In [ ]:
# Import relevant libraries
import os
import pandas as pd

from SALib.analyze.sobol import analyze
import matplotlib.pyplot as plt
import pickle
import pandas as pd

from comparative_gsa.utils import helpers

import torch

from autoemulate.core.sensitivity_analysis import SensitivityAnalysis
from autoemulate.core.sensitivity_analysis import _sobol_results_to_df 

figsize = (9, 5)

In [ ]:
param_path = '../inputs/parameters_naghavi_constrained_fixed_T_v_tot_v_ref_lower_k_pas_further.json'
# Get the filename from the path, without extension
param_filename = os.path.splitext(os.path.basename(param_path))[0]

n_saltelli_samples = [2048, 4096]
n_model_evals      = [61440, 122880]

In [ ]:
data_type = 'simulations' # 'simulations' or 'emulations'

for i_n_saltelli_samples, i_n_model_evals in zip(n_saltelli_samples, n_model_evals):
    print(f'Processing {i_n_saltelli_samples} samples and {i_n_model_evals} model evaluations...')

    simulation_out_path = f'../outputs/simulations_for_sa/n_samples_{i_n_saltelli_samples}_n_evals_{i_n_model_evals}_{param_filename}/'

    # Load the problem pickle file
    problem_path = os.path.join(simulation_out_path, 'problem.pkl')

    with open(problem_path, 'rb') as f:
        problem = pickle.load(f)

    # Load the simulation summary results
    simulations_summary = pd.read_csv(os.path.join(simulation_out_path, f'simulations_summary.csv'))

    # Load the parameter values
    parameters_csv = pd.read_csv(os.path.join(simulation_out_path,'saltelli_samples.csv'))

    # Turn x into a pytorch tensor, for emulators to predict on
    x = torch.tensor(parameters_csv.values, dtype=torch.float32)

    # Loop over each feature
    for i_output_feature in simulations_summary.columns:

        print(f'Analysing {i_output_feature} with {data_type}')

        Y_feature = simulations_summary[i_output_feature].values
       
        # Do the sobol_analyse for GSA
        sobol_indices = analyze(problem, Y_feature, calc_second_order=True)
        
        # Transform to a dataframe  
        sobol_df = _sobol_results_to_df({i_output_feature: sobol_indices})

        # Swap S1 ST due to bug
        print('Warning, swapping S1 and ST because of a bug in Autoemulate plotting')
        sobol_df = helpers.swap_s1_st(sobol_df=sobol_df)

        # Save the sobol_indices and sobol df
        save_dir = os.path.join('../outputs/sa_results/'
                                f'n_samples_{i_n_saltelli_samples}_n_evals_{i_n_model_evals}_{param_filename}/',
                                i_output_feature,
                                data_type)

        os.makedirs(save_dir, exist_ok=True)
        sobol_df.to_csv(os.path.join(save_dir,'sobol_df.csv'), index=False)

        # Plot the figures and save
        fig_s1 = SensitivityAnalysis.plot_sobol(sobol_df, index="S1", figsize=figsize) 
        fig_s1.savefig(os.path.join(save_dir, f's1_indices.png'), 
                dpi=300, bbox_inches='tight')
        plt.close(fig_s1)  # Close the figure to free memory
        fig_st = SensitivityAnalysis.plot_sobol(sobol_df, index="ST", figsize=figsize) 
        fig_st.savefig(os.path.join(save_dir, f'st_indices.png'), 
                dpi=300, bbox_inches='tight')    
        plt.close(fig_st)  # Close the figure to free memory        